In [53]:
import pandas as pd
from pathlib import Path

In [54]:
data_dir = Path(".")
csv_dir = data_dir / "csv"
json_dir = data_dir / "json"

if not json_dir.exists():
    json_dir.mkdir()

In [55]:
csv_files = csv_dir.glob("*.csv")

In [56]:
for file in csv_files:
    df = pd.read_csv(csv_dir / file.name)
    df.set_index(df.columns[0], inplace=True)
    df.fillna("", inplace=True)
    display(df.head(2))
    df.to_json(json_dir / f"{file.stem}.json", orient='index')

,2800,2632,2802,2804,2806
Unnamed: 0,,,,,
Bellville,,,,,
Oosterzee,06:54,07:11,09:04,11:14,13:24


,501,503,505,507,509,511,513
Unnamed: 0,,,,,,,
Cape Town,,,,,,,
Woodstock,04:57,06:27,07:39,08:57,10:07,11:22,13:12


,2501,2701,2505,2507,2703,2705,2707,2709,3503,2711,...,3511,2521,3201,3401,3403,3405,3407,3409,3413,3501
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Cape Town,,,,,,,,,,,...,,,,,,,,,,
Woodstock,,,,,,,,,,,...,,,,,,,,,,


,2501,2301,2503,2303,2505,2305,2507,2307,2801,2803,2805,2807,2661,3501,3503,3505,3507,3509
Unnamed: 0,,,,,,,,,,,,,,,,,,
Cape Town,,,,,,,,,,,,,,,,,,
Woodstock,05:13,06:08,07:03,08:13,09:03,10:23,11:08,12:13,,,,,,,,,,


,101,103,105,107,109,111,113,115,117,119
Unnamed: 0,,,,,,,,,,
Cape Town,,,,,,,,,,
Woodstock,05:23,06:03,06:43,07:23,08:03,09:03,10:03,11:03,12:03,13:33


...

,2801,2803,2805,2807,2809,2811,2813,2815,3207,2817,2677
Unnamed: 0,,,,,,,,,,,
Cape Town,,,,,,,,,,,
Esplanade,,,,,,,,,,,


,100,102,104,106,108,110,112,114,116,118,...,140,142,144,146,148,150,152,154,156,158
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Fish Hoek,,,,,,,,,,,...,,,,,,,,,,
Kalk Bay,04:49,05:14,05:59,06:19,06:39,06:59,07:19,07:39,07:59,08:14,...,15:39,15:59,16:19,16:39,16:59,17:19,17:39,17:59,18:19,18:39


,500,502,504,506,508,510,512,514,516,518,520,522,524,526,528,530,532
Unnamed: 0,,,,,,,,,,,,,,,,,
Retreat,,,,,,,,,,,,,,,,,
Heathfield,05:12,05:52,06:27,06:47,07:32,08:12,09:02,09:22,10:02,11:44,12:44,15:07,16:12,16:27,17:27,18:02,18:47


,200,202,204,206,208,210,212,214
Unnamed: 0,,,,,,,,
Simonstown,,,,,,,,
Glencairn,05:31,06:51,07:51,08:51,09:51,10:51,11:51,12:51


,9500,9502,9504,9506,9508,9400,9402,9404,9406,9408,...,9910,9912,9914,9916,9918,9002,9004,9006,9008,9010
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
Bellville,,,,,,,,,,,...,,,,,,,,,,
Sarepta,,,,,,,,,,,...,,,,,,06:01,08:06,10:51,15:26,17:26
